In [1]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from keras.applications import InceptionV3
from tensorflow.keras import layers, models

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

In [4]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=x)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=1,
    restore_best_weights=True
)
model.fit(x = X_train, y= y_train, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1875/1875 [==============================] - 509s 269ms/step - loss: 0.5089 - accuracy: 0.8187
Epoch 2/10
1875/1875 [==============================] - 502s 268ms/step - loss: 0.3858 - accuracy: 0.8587
Epoch 3/10
1875/1875 [==============================] - 503s 269ms/step - loss: 0.3515 - accuracy: 0.8707
Epoch 4/10
1875/1875 [==============================] - 507s 270ms/step - loss: 0.3250 - accuracy: 0.8790
Epoch 5/10
1875/1875 [==============================] - 506s 270ms/step - loss: 0.3054 - accuracy: 0.8852
Epoch 6/10
1875/1875 [==============================] - 508s 271ms/step - loss: 0.2873 - accuracy: 0.8920
Epoch 7/10
1875/1875 [==============================] - 501s 267ms/step - loss: 0.2699 - accuracy: 0.8982
Epoch 8/10
1875/1875 [==============================] - 495s 264ms/step - loss: 0.2601 - accuracy: 0.9014
Epoch 9/10
1875/1875 [==============================] - 496s 264ms/step - loss: 0.2459 - accuracy: 0.9061
Epoch 10/10
1875/1875 [=======================

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 85s - loss: 0.4026 - accuracy: 0.8660 - 85s/epoch - 271ms/step

Test accuracy: 0.8659999966621399


In [8]:
model.save('NN/inception_fashion_mnist.h5')